## Data Preparation

In [37]:
import pandas as pd

df = pd.read_csv("hf://datasets/bitext/Bitext-media-llm-chatbot-training-dataset/bitext-media-llm-chatbot-training-dataset.csv")
df

,instruction,intent,category,tags,response
0,wanna pseak with fucking customer support wher...,customer_service,CONTACT,BCILQWZ,To reach out to our customer service departmen...
1,I want to send an email to fucking client serv...,customer_service,CONTACT,BCILPWZ,To contact our customer service department via...
2,can ya help me send an email to fucking custom...,customer_service,CONTACT,BILQWZ,To initiate communication with our customer se...
3,"I want to call fucking client service, will yo...",customer_service,CONTACT,BCILPWZ,To reach out to our customer service departmen...
4,"I want to talk to customer service, how oculd ...",customer_service,CONTACT,BCILPZ,To reach our customer service department via e...
...,...,...,...,...,...
24622,"I need to see the fucking subscription fees, h...",subscription_prices,SUBSCRIPTION,BCILW,"To verify your subscription details, please ad..."
24623,I'd like to see the prices of the subscription...,subscription_prices,SUBSCRIPTION,BCIPQZ,"To verify your subscription details, please ad..."
24624,I would like to know more about the subscripti...,subscription_prices,SUBSCRIPTION,BCPZ,"To review your subscription details, please ad..."
24625,I would like to know about the fees fo the pla...,subscription_prices,SUBSCRIPTION,BCILPZ,"To review your subscription details, please ad..."


In [38]:
df['category'].value_counts()

category
SUBSCRIPTION        8000
SETTINGS            4000
PAYMENT             3000
FUNCTIONING         2627
CONTACT             2000
CONTENT             2000
PROGRAM_SCHEDULE    2000
PLATFORM            1000
Name: count, dtype: int64

In [39]:
df[df['category'] == 'CONTENT'].sample(3).to_dict(orient='records')

[{'instruction': 'need to notify of fuckiung inappropiate content where can i do it',
  'intent': 'report_inappropiate_content',
  'category': 'CONTENT',
  'tags': 'BCILQWZ',
  'response': 'To address inappropriate content observed on our platform, please adhere to the following instructions:\n\n1. Visit {{WEBSITE_URL}}.\n2. Identify the relevant content that you intend to report.\n3. Click on the {{REPORT_SECTION}}, typically found in proximity to the content (look for the {{FLAG_ICON}}).\n4. Choose the {{REPORT_INAPPROPRIATE_CONTENT_OPTION}} from the available options in the menu.\n5. Select the reason that best corresponds to your report from the choices provided.\n6. If needed, offer any supplementary information to aid in your report and then submit.\n\nAfter your report has been submitted, our team will assess the situation and take appropriate measures in alignment with our community standards.'},
 {'instruction': 'i have to report inappropiate content could uhelp me',
  'intent

In [40]:
# Select rows where the category is "CONTENT"

# Select a random 10% of the filtered data
df = df.sample(frac=0.21, random_state=1)  # 10% sample
df = df[df['category'] == 'CONTENT']



In [41]:
df.head()

,instruction,intent,category,tags,response
2442,how can i report copyright violation,report_copyright_infringement,CONTENT,BILQ,"To report copyright infringement, please adher..."
2032,need to report fucking copyright violation how...,report_copyright_infringement,CONTENT,BCILQW,To facilitate a conversation with a customer s...
2868,need to report copyright infringement how do i...,report_copyright_infringement,CONTENT,BCIQ,To initiate a report for copyright infringemen...
2999,can uhelp me submitting a notification of copy...,report_copyright_infringement,CONTENT,BILQZ,"To address copyright infringement concerns, pl..."
3310,I would like to notify of fucking explicit con...,report_inappropiate_content,CONTENT,BCLPW,To address concerns related to inappropriate c...


In [42]:
df.shape

(424, 5)

In [43]:
df.to_csv("../Data/data.csv", index=False)

In [44]:
df.shape

(424, 5)

In [ ]:
import json

# Initialize an empty dictionary to hold the transformed data
data = []

# Group by 'category'
for category, group_df in df.groupby('category'):
    documents = []
    for _, row in df.iterrows():
        document = {
            "intent": row['intent'],
            "question": row['instruction'],
            "response": row['response']
        }
        documents.append(document)
    data.append(
        {
        "category": category,
        "documents": documents
    })

# Convert the dictionary to a list of values and then to a JSON file
with open('../Data/documents.json', 'w', encoding='utf-8') as jsonf:
    json.dump(data, jsonf, indent=2, ensure_ascii=False)

print("JSON file saved as documents.json")

JSON file saved as documents.json


In [46]:
import json

with open('../Data/documents.json', 'rt') as f_in:
    documents_raw = json.load(f_in)

In [47]:
documents = []

for category in documents_raw:
    category_name = category['category']

    for doc in category['documents']:
        doc['category'] = category_name
        documents.append(doc)

In [48]:
import hashlib

def generate_document_id(doc):
    combined = f"{doc['category']}-{doc['question']}-{doc['response'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [49]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [50]:
documents[3]

{'intent': 'report_copyright_infringement',
 'question': 'can uhelp me submitting a notification of copyright  infringement',
 'response': 'To address copyright infringement concerns, please adhere to the following guidelines:\n\n1. Access our website at {{WEBSITE_URL}}.\n2. Go to the {{COPYRIGHT_SECTION}} area.\n3. Find and select the {{REPORT_COPYRIGHT_INFRINGEMENT_OPTION}} choice.\n4. Complete the necessary fields, providing comprehensive details regarding the infringement.\n5. Submit the completed form for our assessment.\n\nYour submission will be reviewed, and we will take the requisite steps in line with our policies.',
 'category': 'CONTENT',
 'id': '1e254822'}

In [51]:
len(documents)

424

In [52]:
with open('../Data/documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)